# C4.5算法

## np.nanmax(), np.nanmin()
- https://blog.csdn.net/weixin_36670529/article/details/116498936
- 忽略数据中的nan值计算最大，最小值

## np.linspace()
- 通过定义均匀间隔创建数值序列。其实，需要指定间隔起始点、终止端，以及指定分隔值总数（包括起始点和终止点）
- https://blog.csdn.net/neweastsun/article/details/99676029

## pd.cat()
- https://blog.csdn.net/weixin_45144170/article/details/105698517
- 将序列中的元素连接在一起

## numpy.array 的shape
- https://blog.csdn.net/lian740930980/article/details/127325506
- numpy 创建的数组都有一个shape属性，它是一个元组，返回各个维度的维数
- 当y是一个两行三列的二维数组，y.shape[0]代表行数，y.shape[1]代表列数。

## np.unique()
- 去除其中重复的元素 ，并按元素由小到大 返回一个新的无元素重复的元组或者列表
- https://blog.csdn.net/weixin_44211968/article/details/123772201
- numpy.unique(arr, return_index, return_inverse, return_counts)
- arr：输入数组，如果不是一维数组则会展开
- return_index：如果为 true，返回新列表元素在旧列表中的位置（下标），并以列表形式存储。
- return_inverse：如果为true，返回旧列表元素在新列表中的位置（下标），并以列表形式存储。
- return_counts：如果为 true，返回去重数组中的元素在原数组中的出现次数。

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 读取数据
data = pd.read_csv('data/titanic/train.csv')

c:\Users\OneRaise\miniconda3\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\OneRaise\miniconda3\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\OneRaise\miniconda3\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# 删掉编号，姓名，船票
data.drop(columns=['PassengerId', 'Name', 'Ticket'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [8]:
# 将连续特征分组
feat_ranges = {}
cont_feat = ['Age', 'Fare']  # Fare为船票价格
bins = 10  # 分类点数

for feat in cont_feat:
    min_val = np.nanmin(data[feat])
    max_val = np.nanmax(data[feat])
    feat_ranges[feat] = np.linspace(min_val, max_val, bins).tolist()
feat_ranges

{'Age': [0.42,
  9.262222222222222,
  18.104444444444447,
  26.94666666666667,
  35.78888888888889,
  44.63111111111111,
  53.473333333333336,
  62.31555555555556,
  71.15777777777778,
  80.0],
 'Fare': [0.0,
  56.925466666666665,
  113.85093333333333,
  170.7764,
  227.70186666666666,
  284.62733333333335,
  341.5528,
  398.4782666666666,
  455.4037333333333,
  512.3292]}

In [9]:
# 将离散特征的值转化为整数
# Pclass为仓位等级，SibSp为登船的兄弟姐妹的数量，Parch为登船的父母子女的数量
# Cabin为船舱编号，Embarked为登船港口
cat_feat = ['Sex', 'Pclass', 'SibSp', 'Parch', 'Cabin', 'Embarked']
for feat in cat_feat:
    data[feat] = data[feat].astype('category')  # 数据格式转化为分类格式
    print(f'{feat}:{data[feat].cat.categories}')  # 查看类别
    data[feat] = data[feat].cat.codes.to_list()  # 将类别按顺序转换为整数
    ranges = list(set(data[feat]))
    ranges.sort()
    feat_ranges[feat] = ranges

Sex:Index(['female', 'male'], dtype='object')
Pclass:Index([1, 2, 3], dtype='int64')
SibSp:Index([0, 1, 2, 3, 4, 5, 8], dtype='int64')
Parch:Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')
Cabin:Index(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31', 'A32',
       ...
       'E8', 'F E69', 'F G63', 'F G73', 'F2', 'F33', 'F38', 'F4', 'G6', 'T'],
      dtype='object', length=147)
Embarked:Index(['C', 'Q', 'S'], dtype='object')


In [10]:
feat_ranges

{'Age': [0.42,
  9.262222222222222,
  18.104444444444447,
  26.94666666666667,
  35.78888888888889,
  44.63111111111111,
  53.473333333333336,
  62.31555555555556,
  71.15777777777778,
  80.0],
 'Fare': [0.0,
  56.925466666666665,
  113.85093333333333,
  170.7764,
  227.70186666666666,
  284.62733333333335,
  341.5528,
  398.4782666666666,
  455.4037333333333,
  512.3292],
 'Sex': [0, 1],
 'Pclass': [0, 1, 2],
 'SibSp': [0, 1, 2, 3, 4, 5, 6],
 'Parch': [0, 1, 2, 3, 4, 5, 6],
 'Cabin': [-1,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,

In [11]:
# 将所有缺省值替换为-1
data.fillna(-1, inplace=True)
for feat in feat_ranges.keys():
    feat_ranges[feat] = [-1] + feat_ranges[feat]

In [12]:
feat_ranges

{'Age': [-1,
  0.42,
  9.262222222222222,
  18.104444444444447,
  26.94666666666667,
  35.78888888888889,
  44.63111111111111,
  53.473333333333336,
  62.31555555555556,
  71.15777777777778,
  80.0],
 'Fare': [-1,
  0.0,
  56.925466666666665,
  113.85093333333333,
  170.7764,
  227.70186666666666,
  284.62733333333335,
  341.5528,
  398.4782666666666,
  455.4037333333333,
  512.3292],
 'Sex': [-1, 0, 1],
 'Pclass': [-1, 0, 1, 2],
 'SibSp': [-1, 0, 1, 2, 3, 4, 5, 6],
 'Parch': [-1, 0, 1, 2, 3, 4, 5, 6],
 'Cabin': [-1,
  -1,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
 

In [13]:
# 划分训练集与测试集
np.random.seed(2023)
feat_names = data.columns[1:]
label_name = data.columns[0]
# 重排下标后，按照新的下表索引数据
data = data.reindex(np.random.permutation(data.index))
ratio = 0.8
split = int(ratio * len(data))
train_x = data[:split].drop(columns=['Survived']).to_numpy()
train_y = data['Survived'][:split].to_numpy()
test_x = data[split:].drop(columns=['Survived']).to_numpy()
test_y = data['Survived'][split:].to_numpy()
print('训练集大小：', len(train_x))
print('测试集大小：', len(test_x))
print('特征数：', train_x.shape[1])

训练集大小： 712
测试集大小： 179
特征数： 8


# C4.5算法的实现

In [14]:
class Node:
    def __init__(self):
        # 内部节点的feat表示用来分类的特征编号，其数字与数据中的顺序对应
        # 叶节点的feat表示该节点对应的分类结果
        self.feat = None
        # 分类值列表，表示按照其中的值向子节点分类
        self.split = None
        # 子节点列表，叶节点的child为空
        self.child = []

class DecisionTree:
    def __init__(self, X, Y, feat_ranges, lbd):
        self.root = Node()
        self.X = X
        self.Y = Y
        self.feat_ranges = feat_ranges  # 特征取值范围
        self.lbd = lbd  # 正则化约束强度
        self.eps = 1e-8  # 防止数学错误log(0)和除以0
        self.T = 0  # 记录叶子节点个数
        self.ID3(self.root, self.X, self.Y)

    # 工具函数，计算 a * log(a)
    def aloga(self, a):
        return a * np.log2(a + self.eps)
    
    # 计算某个子数据集的熵
    def entropy(self, Y):
        cnt = np.unique(Y, return_counts=True)[1]  # 统计每个类别出现的次数
        N = len(Y)
        ent = -np.sum([self.aloga(Ni / N) for Ni in cnt])
        return ent
    
    # 用feat <= val划分数据集的信息增益
    def info_gain(self, X, Y, feat, val):
        # 划分前的熵
        N = len(Y)
        if N == 0:
            return 0
        HX = self.entropy(Y)
        HXY = 0  # H(X|Y)
        # 分别计算val两侧的H(X|Y)
        Y_l = Y[X[:, feat] <= val]
        HXY += len(Y_l) / len(Y)  * self.entropy(Y_l)
        Y_r = Y[X[:, feat] > val]
        HXY += len(Y_r) / len(Y) * self.entropy(Y_r)
        return HX - HXY

    # 计算feat <= val本身的复杂度
    def entropy_YX(self, X, Y, feat, val):
        HYX = 0
        N = len(Y)
        if N == 0:
            return 0
        Y_l = Y[X[:, feat] <= val]
        HYX += -self.aloga(len(Y_l) / N)
        Y_r = Y[X[:, feat] > val]
        HYX += -self.aloga(len(Y_r) / N)
        return HYX
    
    # 计算信息增益率
    def info_gain_ratio(self, X, Y, feat, val):
        IG = self.info_gain(X, Y, feat, val)
        HYX = self.entropy_YX(X, Y, feat, val)
        return IG/HYX
    
    # ID3算法递归分类节点，构造决策树
    def ID3(self, node, X, Y):
        # 判断是否已经完成分类
        if len(np.unique(Y)) == 1:
            node.feat = Y[0]
            self.T += 1
            return

        # 寻找最优分类特征和分类点
        best_IGR = 0
        best_feat = None
        best_val = None
        for feat in range(len(feat_names)):
            for val in self.feat_ranges[feat_names[feat]]:
                IGR = self.info_gain_ratio(X, Y, feat, val)
                if IGR > best_IGR:
                    best_IGR = IGR
                    best_feat = feat
                    best_val = val
        # 计算用best_feat <= best_val分类带来的代价函数的变化
        # 由于分裂节点只涉及该局部，我们只需要计算分裂前后该节点的代价函数

        # 当前代价
        cur_cost = len(Y) * self.entropy(Y) + self.lbd

        # 分裂后的代价，按best_feat的取值分类统计
        # 如果best_feat为None，说明最优的信息增益率为0
        # 再分类也无法增加信息了，因此将new_cost设置为无穷大
        if best_feat is None:
            new_cost = np.inf
        else:
            new_cost = 0
            X_feat = X[:, best_feat]
            # 获取划分后的两部分，计算新的树
            new_Y_l = Y[X_feat <= best_val]
            new_cost += len(new_Y_l) * self.entropy(new_Y_l)
            new_Y_r = Y[X_feat > best_val]
            new_cost += len(new_Y_r) * self.entropy(new_Y_r)
            # 分裂后会有两个叶节点
            new_cost += 2 * self.lbd
            
        if new_cost <= cur_cost:
            # 如果分裂后代价更小，那么执行分裂
            node.feat = best_feat
            node.split = best_val
            l_child = Node()
            l_X = X[X_feat <= best_val]
            l_Y = Y[X_feat <= best_val]
            self.ID3(l_child, l_X, l_Y)
            r_child = Node()
            r_X = X[X_feat > best_val]
            r_Y = Y[X_feat > best_val]
            self.ID3(r_child, r_X, r_Y)
            node.child = [l_child, r_child]
        else:
            # 否则将当前节点上最多的类别作为该节点的类别
            vals, cnt = np.unique(Y, return_counts=True)
            node.feat = vals[np.argmax(cnt)]
            self.T += 1
        
    # 预测新样本的分类
    def predict(self, x):
        node = self.root
        # 从根节点开始向下寻找，到叶节点结束
        while node.split is not None:
            # 判断x应该属于哪个子节点
            if x[node.feat] <= node.split:
                node = node.child[0]
            else:
                node = node.child[1]
        # 到达叶节点，返回类别
        return node.feat
    
    # 计算在样本X，标签Y上的准确率
    def accuracy(self, X, Y):
        correct = 0
        for x, y in zip(X, Y):
            pred = self.predict(x)
            if pred == y:
                correct += 1
        return correct / len(Y)

In [15]:
DT = DecisionTree(train_x, train_y, feat_ranges, lbd=1.0)
print('叶子节点数量：', DT.T)

# 计算在测试集和训练集上的准确率
print('训练集准确率：', DT.accuracy(train_x, train_y))
print('测试集准确率：', DT.accuracy(test_x, test_y))

叶子节点数量： 33
训练集准确率： 0.8525280898876404
测试集准确率： 0.8156424581005587


# sklearn中的决策树

In [16]:
from sklearn import tree
# criterion表示分类依据，max_depth表示树的最大深度
# entropy生成的是C4.5分类树
c45 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6)
c45.fit(train_x, train_y)
# gini生成的是CART分类树
cart = tree.DecisionTreeClassifier(criterion='gini', max_depth=6)
cart.fit(train_x, train_y)

c45_train_pred = c45.predict(train_x)
c45_test_pred = c45.predict(test_x)
cart_train_pred = cart.predict(train_x)
cart_test_pred = cart.predict(test_x)
print('准确率：')
print('训练集：c45：',np.mean(c45_train_pred == train_y),
      'cart:',np.mean(cart_train_pred == train_y))
print('测试集：c45：',np.mean(c45_test_pred == test_y),
      'cart:',np.mean(cart_test_pred == test_y))

准确率：
训练集：c45： 0.8609550561797753 cart: 0.8721910112359551
测试集：c45： 0.8212290502793296 cart: 0.8435754189944135


In [17]:
# PyDotPlus
from six import StringIO
import pydotplus

dot_data = StringIO()
# 导出sklearn中的决策树的可视化数据
tree.export_graphviz(
    c45,
    out_file=dot_data,
    feature_names=feat_names,
    class_names=['non-survival', 'survival'],
    filled=True,
    rounded=True,
    impurity=False
)
# 用pydotplus生成图像
graph = pydotplus.graph_from_dot_data(dot_data.getvalue().replace('\n',''))
graph.write_png('data/tree.png')

True